In [1]:
import findspark
findspark.init()


In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import  types
from pyspark.sql import functions as F
import pandas as pd

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!wget -P /home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
!wc -l /home/Дмитрий/datacamp/dataeng-zoomcamp/week_5_batch_processing/data/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('data/fhvhv_tripdata_2021-01.csv.gz')


In [ ]:
df.head(5)

In [ ]:
df.schema

In [ ]:
# unpack gz file
!gunzip data/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
!head -n 101 data/fhvhv_tripdata_2021-01.csv > data/head.csv

In [ ]:
df_pandas = pd.read_csv('data/head.csv')

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhvhv_tripdata_2021-01.csv.gz')


In [ ]:
df.head(10)

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('data/fhvhv/2021/01/')

In [4]:
# read partition parquet files
df = spark.read.parquet('data/fhvhv/2021/01/')


In [ ]:
# show schema dataframe
df.printSchema()

In [10]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .head(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 1, 16, 47, 20), dropoff_datetime=datetime.datetime(2021, 1, 1, 16, 58, 28), PULocationID=50, DOLocationID=163),
 Row(pickup_datetime=datetime.datetime(2021, 1, 5, 2, 0, 14), dropoff_datetime=datetime.datetime(2021, 1, 5, 2, 19, 39), PULocationID=48, DOLocationID=95),
 Row(pickup_datetime=datetime.datetime(2021, 1, 2, 0, 34, 43), dropoff_datetime=datetime.datetime(2021, 1, 2, 0, 45, 38), PULocationID=63, DOLocationID=77),
 Row(pickup_datetime=datetime.datetime(2021, 1, 2, 16, 20, 11), dropoff_datetime=datetime.datetime(2021, 1, 2, 16, 56, 36), PULocationID=63, DOLocationID=244),
 Row(pickup_datetime=datetime.datetime(2021, 1, 24, 16, 0, 53), dropoff_datetime=datetime.datetime(2021, 1, 24, 16, 7, 40), PULocationID=210, DOLocationID=165)]

In [6]:
df.head(10)

[Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 10, 17, 40, 1), dropoff_datetime=datetime.datetime(2021, 1, 10, 17, 48, 10), PULocationID=97, DOLocationID=25, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 8, 18, 19, 44), dropoff_datetime=datetime.datetime(2021, 1, 8, 18, 55, 57), PULocationID=138, DOLocationID=265, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02876', pickup_datetime=datetime.datetime(2021, 1, 1, 16, 47, 20), dropoff_datetime=datetime.datetime(2021, 1, 1, 16, 58, 28), PULocationID=50, DOLocationID=163, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 1, 15, 17, 50, 8), dropoff_datetime=datetime.datetime(2021, 1, 15, 18, 7, 24), PULocationID=163, DOLocationID=79, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pic

In [22]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_ud', crazy_staff_udf(df.dispatching_base_num)) \
    .select('base_ud', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_ud|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-10|  2021-01-10|          97|          25|
|  e/9ce| 2021-01-08|  2021-01-08|         138|         265|
|  e/b3c| 2021-01-01|  2021-01-01|          50|         163|
|  e/9ce| 2021-01-15|  2021-01-15|         163|          79|
|  e/9ce| 2021-01-12|  2021-01-12|          47|          74|
|  s/acd| 2021-01-05|  2021-01-05|          48|          95|
|  e/b38| 2021-01-02|  2021-01-02|          63|          77|
|  e/9ce| 2021-01-06|  2021-01-06|         238|          41|
|  e/acc| 2021-01-02|  2021-01-02|          63|         244|
|  e/acc| 2021-01-24|  2021-01-24|         210|         165|
|  e/b35| 2021-01-16|  2021-01-16|         113|         143|
|  e/9ce| 2021-01-28|  2021-01-28|          91|          89|
|  e/acc| 2021-01-01|  2021-01-01|         231|         148|
|  e/b33| 2021-01-19|  2

In [16]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-10 17:40:01|2021-01-10 17:48:10|          97|          25|   null|
|           HV0005|              B02510|2021-01-08 18:19:44|2021-01-08 18:55:57|         138|         265|   null|
|           HV0003|              B02876|2021-01-01 16:47:20|2021-01-01 16:58:28|          50|         163|   null|
|           HV0005|              B02510|2021-01-15 17:50:08|2021-01-15 18:07:24|         163|          79|   null|
|           HV0005|              B02510|2021-01-12 17:26:40|2021-01-12 17:57:57|          47|          74|   null|
|           HV0003|              B02765|2021-01-05 02:00:14|2021-01-05 02:19:39|

In [18]:
def crazy_staff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [21]:
crazy_staff_udf = F.udf(crazy_staff, returnType=types.StringType())